# JAXTAM - Just Another X-Ray Timing Module

## Walkthrough

First, import JAXTAM by calling `using JAXTAM`

In [1]:
using JAXTAM

Now, the local user configuration needs to be set up.

## Configuration

In [2]:
JAXTAM.config()

INFO: Creating config file at: /home/robert/.julia/v0.6/JAXTAM/user_configs.jld2


Dict{Any,Any} with 1 entry:
  :_config_edit_date => 2018-07-06T11:31:55.284

Add in the missions. Mission configurations are defined by a `MissionDefinition` type:


```
mutable struct MissionDefinition
    name::String
    url::String
    path::String
    path_obs::Function
    path_cl::Function
    path_uf::Function
    path_rmf::String
end
```

These types contain the mission `name`, an `url` to the HEASARC master table for that mission, a `path` to the local folder the data will be stored in, three path functions (`path_obs`, `path_cl`, and `path_uf`) which return the path to the relevant folders/files, given a row from the master table. Functions have to be used for this as some more complex path structures exist, like for NICER which uses the observation date for the folders.

JAXTAM includes default missions, if a mission name is included in these defaults then only the local `path` is required. This speeds up adding new missions:

In [3]:
JAXTAM.config(:nicer, "/media/robert/HVS1/heasrac/nicer/")

INFO: nicer found in defaults
Using /media/robert/HVS1/heasrac/nicer/ as path


Dict{Any,Any} with 2 entries:
  :nicer             => JAXTAM.MissionDefinition("nicer", "https://heasarc.gsfc…
  :_config_edit_date => 2018-07-06T11:32:04.008

In [4]:
JAXTAM.config(:nustar, "/media/robert/HVS1/heasrac/nustar/")

INFO: nustar found in defaults
Using /media/robert/HVS1/heasrac/nustar/ as path


Dict{Any,Any} with 3 entries:
  :nicer             => JAXTAM.MissionDefinition("nicer", "https://heasarc.gsfc…
  :_config_edit_date => 2018-07-06T11:32:04.924
  :nustar            => JAXTAM.MissionDefinition("nustar", "https://heasarc.gsf…

A default mission can also be set in the configuration:

In [5]:
JAXTAM.config(:default, :nicer)

INFO: Setting default mission to nicer


Dict{Any,Any} with 4 entries:
  :nicer             => JAXTAM.MissionDefinition("nicer", "https://heasarc.gsfc…
  :default           => :nicer
  :_config_edit_date => 2018-07-06T11:32:05.1
  :nustar            => JAXTAM.MissionDefinition("nustar", "https://heasarc.gsf…

Removing missions is simple and can be done with `config_rm(mission_name`.

## Master Tables

As the url to the master table is included in the mission definitions, downloading the master tables is simple:

In [6]:
JAXTAM.master(:nicer)[1:2, :]

	/media/robert/HVS1/heasrac/nicer/master.tdat 
	/media/robert/HVS1/heasrac/nicer/master.jld2
INFO: Download master files from `https://heasarc.gsfc.nasa.gov/FTP/heasarc/dbase/tdat_files/heasarc_nicermastr.tdat.gz`? (y/n)
INFO: Downloading latest master catalog
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  358k  100  358k    0     0   220k      0  0:00:01  0:00:01 --:--:--  220k



7-Zip [64] 9.20  Copyright (c) 1999-2010 Igor Pavlov  2010-11-18
p7zip Version 9.20 (locale=en_GB.UTF-8,Utf16=on,HugeFiles=on,4 CPUs)

Processing archive: /media/robert/HVS1/heasrac/nicer/master.tdat

Extracting  master

Everything is Ok

Size:       2729320
Compressed: 366731


INFO: Loading /media/robert/HVS1/heasrac/nicer/master.tdat
INFO: Saving /media/robert/HVS1/heasrac/nicer/master.jld2


,name,ra,dec,lii,bii,time,end_time,obsid,exposure,time_awarded,num_fpm,processing_status,processing_date,public_date,processing_version,num_processed,caldb_version,software_version,prnb,abstract,subject_category,category_code,pi_lname,pi_fname,cycle,obs_type,title,remarks
1,1E_0102.2-7219,15.98982,-72.064019999999999,301.56842509000001,-45.031067120000003,2017-06-19T19:01:25,2017-06-19T19:13:02,0010010101,3.0,0.0,52,PROCESSED,2018-03-02T21:41:19,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing
2,1E_0102.2-7219,16.015969999999999,-72.062550000000002,301.55691327,-45.031977480000002,2017-06-22T00:42:42,2017-06-22T08:39:55,0010010104,3.0,0.0,52,PROCESSED,2018-03-02T21:50:08,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing


Once the table has been downloaded, repeated calls to `master` will use the current version of the table. To update the table, simply call `master_update`

### Append Table

For sanity, the master table is not modified (apart from dates being converted from MJD to the standard yyyy-mm-ddThh:mm:ss format), so an `append` table is used to store new variables.

In [7]:
JAXTAM.append()[1:2, :]

INFO: Using default mission - nicer
INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2
INFO: Saving /media/robert/HVS1/heasrac/nicer/append.jld2


,obsid,publicity,obs_path,event_uf,event_cl,downloaded
1,0010010101,false,/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101,"(""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu0_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu1_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu2_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu3_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu4_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu5_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_uf/ni0010010101_0mpu6_uf.evt"")","(""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010101/xti/event_cl/ni0010010101_0mpu7_cl.evt"",)",false
2,0010010104,false,/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104,"(""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu0_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu1_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu2_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu3_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu4_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu5_uf.evt"", ""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_uf/ni0010010104_0mpu6_uf.evt"")","(""/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201706a/obs/2017_06/0010010104/xti/event_cl/ni0010010104_0mpu7_cl.evt"",)",false


The append table contains file and folder paths for each observation, which are found using the path functions contained in the mission definition, as well as a publicity flag and a downloaded flag. This table can easily be expanded on as required.

To automatically load a joined version of the master and append tables, call `master_a()`.

### Master Query

For utility, a generic query function exists, allowing users to search for specific column values within the master and append tables.

In [8]:
master_df = JAXTAM.master(:nicer);

INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2


In [9]:
JAXTAM.master_query(master_df, :obsid, "1010010128")

,name,ra,dec,lii,bii,time,end_time,obsid,exposure,time_awarded,num_fpm,processing_status,processing_date,public_date,processing_version,num_processed,caldb_version,software_version,prnb,abstract,subject_category,category_code,pi_lname,pi_fname,cycle,obs_type,title,remarks
1,1E_0102.2-7219,16.009440000000001,-72.031440000000003,301.55670425,-45.063152170000002,2017-12-03T17:09:18,2017-12-03T18:52:00,1010010128,72.0,60000.0,52,VALIDATED,2018-03-03T03:38:41,2018-03-04T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,1010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,1,CAL,CALIBRATION,missing


In [10]:
JAXTAM.master_query(master_df, :subject_category, "MAGNETAR")[1:2, :]

,name,ra,dec,lii,bii,time,end_time,obsid,exposure,time_awarded,num_fpm,processing_status,processing_date,public_date,processing_version,num_processed,caldb_version,software_version,prnb,abstract,subject_category,category_code,pi_lname,pi_fname,cycle,obs_type,title,remarks
1,1E_1048.1-5937,162.52000000000001,-59.921030000000002,288.26640995999998,-0.55070085999999996,2017-06-22T17:19:29,2017-06-22T22:25:42,0020240101,0.0,0.0,52,PROCESSED,2018-03-03T05:15:30,2099-12-31T00:00:00,l0-master_20180226,2,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0020,Observations dedicated to the Magnetar science working group,MAGNETAR,2,GENDREAU,KEITH C.,0,NOR,MAGNETAR WORKING GROUP,missing
2,1E_1048.1-5937,162.54390000000001,-59.890210000000003,288.26326461000002,-0.51778281000000004,2018-01-30T14:50:43,2018-01-30T15:14:04,1020240126,729.0,60000.0,52,VALIDATED,2018-03-05T12:33:49,2018-03-04T00:00:00,l0-master_20180226,4,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,1020,Observations dedicated to the Magnetar science working group,MAGNETAR,2,GENDREAU,KEITH C.,1,NOR,MAGNETAR WORKING GROUP,missing


Public only query functions are also available, in this case we see NICER has 5850 public observations.

In [11]:
size(JAXTAM.master_query_public(:nicer))

INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2


(5850, 28)

In [12]:
size(JAXTAM.master_query_public(master_df, :subject_category, "MAGNETAR"))

(1353, 28)

With 1353 of them being magnetars.

## Downloading Observations

Downloading observations is made to be simple and intuitive, simply give an observation ID and the observation will be downloaded, respecting the FTP folder structure, into the mission path specified in the configuration file:

In [13]:
JAXTAM.download(:nicer, "1010010192")

INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2
INFO: heasarc.gsfc.nasa.gov:/.nicer_archive/.nicer_201805a/obs/2018_05/1010010192 --> /media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192


`lftp heasarc.gsfc.nasa.gov -e 'mirror "/.nicer_archive/.nicer_201805a/obs/2018_05/1010010192" "/media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192" --parallel=10 --only-newer && exit'`


This is done via `lftp` using 10 parallel connections for speed.

The download command can be paired with query to queue up multiple downloads, for example downloading all public observations would be:

In [ ]:
#JAXTAM.download(:nicer, JAXTAM.master_query_public(:nicer)[:obsid])

## Analysis

### Reading Observation to Feather

Reading an observation in must be done before any analysis is started. This function converts the FITS files into feather ones for faster i/o operations, and stores a few relevant fits comments:

In [14]:
instrument_data = JAXTAM.read_cl(:nicer, "1010010192")

INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2
INFO: Loading /media/robert/HVS1/heasrac/nicer/append.jld2


INFO: Found: /media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192/xti/event_cl/ni1010010192_0mpu7_cl.evt.gz
INFO: Found 1 file(s) for 1010010192
INFO: Loading /media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192/xti/event_cl/ni1010010192_0mpu7_cl.evt.gz
INFO: Saving XTI


Dict{Symbol,JAXTAM.InstrumentData} with 1 entry:
  :XTI => JAXTAM.InstrumentData("1010010192", :XTI, 17282×13 DataFrames.DataFra…

The observations are stored in a dictionary, with instruments as the dictionary key. This is done to simplify the cases where a mission has multiple instruments, such as NuSTAR:

In [15]:
JAXTAM.read_cl(:nustar, "10002008001")

INFO: Loading /media/robert/HVS1/heasrac/nustar/master.jld2
INFO: Loading /media/robert/HVS1/heasrac/nustar/append.jld2
INFO: Loading 10002008001: FPMA from /media/robert/HVS1/heasrac/nustar/nustar/nustar_archive/10002008001/JAXTAM/
INFO: Loading 10002008001: FPMB from /media/robert/HVS1/heasrac/nustar/nustar/nustar_archive/10002008001/JAXTAM/


Dict{Symbol,JAXTAM.InstrumentData} with 2 entries:
  :FPMB => JAXTAM.InstrumentData("10002008001", :FPMB, 524403×27 DataFrames.Dat…
  :FPMA => JAXTAM.InstrumentData("10002008001", :FPMA, 571026×27 DataFrames.Dat…

The data is stored in an `InstrumentData` type, which currently contains:

```
struct InstrumentData
    obsid::String
    instrument::Symbol
    events::DataFrame
    gtis::DataFrame
    start::Number
    stop::Number
end
```

However, it is planned to read all of the header comments in the fits file and store them in a meta variable for later use if required.

In [16]:
instrument_data[:XTI]

JAXTAM.InstrumentData("1010010192", :XTI, 17282×13 DataFrames.DataFrame. Omitted printing of 6 columns
│ Row   │ TIME      │ RAWX │ RAWY │ PHA │ PHA_FAST │ DET_ID │ DEADTIME   │
├───────┼───────────┼──────┼──────┼─────┼──────────┼────────┼────────────┤
│ 1     │ 1.3738e8  │ 0x03 │ 0x01 │ 481 │ 500      │ 0x23   │ 2.26373f-5 │
│ 2     │ 1.3738e8  │ 0x02 │ 0x04 │ 561 │ 573      │ 0x34   │ 2.26373f-5 │
│ 3     │ 1.3738e8  │ 0x05 │ 0x00 │ 613 │ 512      │ 0x10   │ 2.26373f-5 │
│ 4     │ 1.3738e8  │ 0x02 │ 0x06 │ 888 │ 807      │ 0x32   │ 2.26373f-5 │
│ 5     │ 1.3738e8  │ 0x02 │ 0x00 │ 561 │ 515      │ 0x25   │ 2.26373f-5 │
│ 6     │ 1.3738e8  │ 0x02 │ 0x06 │ 537 │ 465      │ 0x32   │ 2.26373f-5 │
│ 7     │ 1.3738e8  │ 0x06 │ 0x04 │ 720 │ 741      │ 0x0c   │ 2.26373f-5 │
│ 8     │ 1.3738e8  │ 0x03 │ 0x05 │ 355 │ -32768   │ 0x29   │ 1.51949f-5 │
│ 9     │ 1.3738e8  │ 0x03 │ 0x02 │ 487 │ 464      │ 0x2c   │ 2.26373f-5 │
│ 10    │ 1.3738e8  │ 0x00 │ 0x06 │ 482 │ -32768   │ 0x3d   │ 1.51949f-5

### Energy Calibration

The PI channels can be converted into energies quite easily:

In [17]:
JAXTAM.calibrate(:nicer, "1010010192")

INFO: Loading /media/robert/HVS1/heasrac/nicer/master.jld2
INFO: Loading /media/robert/HVS1/heasrac/nicer/append.jld2
INFO: Loading EVENTS for 1010010192 from /media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192/JAXTAM/
INFO: Loading XTI EVENTS
INFO: Loading 1010010192: XTI from /media/robert/HVS1/heasrac/nicer/nicer_archive/nicer_201805a/obs/2018_05/1010010192/JAXTAM/
INFO: Saving XTI CALIB energy


Dict{Symbol,JAXTAM.InstrumentData} with 1 entry:
  :XTI => JAXTAM.InstrumentData("1010010192", :XTI, 17282×14 DataFrames.DataFra…

This simply adds an extra column to the `InstrumentData.events` DataFrame which contains the energy of that event.

### Light Curves

Binning the events into a light curve can also be done easily:

In [18]:
JAXTAM._lcurve(instrument_data[:XTI], 1)

((0.0:1.0:5942.0,),   [57  ]  =  25
  [58  ]  =  31
  [59  ]  =  30
  [60  ]  =  25
  [61  ]  =  26
  [62  ]  =  28
  [63  ]  =  27
  [64  ]  =  23
  [65  ]  =  26
  [66  ]  =  33
          ⋮
  [5903]  =  32
  [5904]  =  2
  [5906]  =  30
  [5907]  =  27
  [5908]  =  20
  [5909]  =  34
  [5910]  =  25
  [5911]  =  21
  [5912]  =  4
  [5927]  =  22
  [5928]  =  5)

This currently returns the time edges, as well as a sparese array of the counts.